In [ ]:
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn import model_selection
from sklearn import metrics
import numpy as np
import pandas as pd
import catboost
import gc
from sklearn import datasets
import lightgbm as lgb

In [ ]:
path = './'
train_application = pd.read_csv(path + 'train/train_Application.csv')
train_history_application = pd.read_csv(path + 'train/train_History_Application.csv')
train_history_payment = pd.read_csv(path + 'train/train_History_Payment.csv')
train_label = pd.read_csv(path + 'train/train_label.csv')
train_personas = pd.read_csv(path + 'train/train_Personas.csv')

print('train_application.shape:', train_application.shape)
print('train_history_application.shape: ', train_history_application.shape)
print('train_history_payment.shape: ', train_history_payment.shape)
print('train_label.shape: ', train_label.shape)
print('train_personas.shape: ', train_personas.shape)

# load test data
A_application = pd.read_csv(path + 'A/A_Application.csv')
A_history_application = pd.read_csv(path + 'A/A_History_Application.csv')
A_history_payment = pd.read_csv(path + 'A/A_History_Payment.csv')
A_personas = pd.read_csv(path + 'A/A_Personas.csv')

print('A_application.shape', A_application.shape)
print('A_history_application.shape:', A_history_application.shape)
print('A_history_payment.shape:', A_history_payment.shape)
print('A_personas.shape: ', A_personas.shape)

# history_application_feature

In [3]:
def encode_category_feature(data, category_name_list):
    """统计类别变量中每个用户在每个类别的数量
       输入原始数 和 类别变量名，返回每个申请编号下的用户每个类别的统计值
    """
    ans = pd.DataFrame(data['申请编号'].unique(), columns=['申请编号'])
    for col_name in category_name_list:
        cols = ['申请编号', '历史申请编号'] + [col_name]
        df_tmp = data.loc[:, cols].copy()
        groupby_col = ['申请编号'] + [col_name]
        df_tmp_groupby = df_tmp.groupby(groupby_col, as_index=True).count()
        df_unstack = df_tmp_groupby.unstack(level=-1)
        new_names = [df_unstack.columns.names[1] + '_' + str(i[1]) for i in df_unstack.columns]
        ans_tmp = pd.DataFrame(data=df_unstack.values, index=df_unstack.index, columns=new_names).reset_index(drop=False)
        
        ans = pd.merge(left=ans, right=ans_tmp, on='申请编号', how='outer')
    ans.fillna(0, inplace=True)
    print('category shape:', ans.shape)
    return ans, category_name_list

def history_application_feature_li(train_history_application_, test_history_application_):
    all_History_Application = pd.concat([train_history_application_, test_history_application_],axis=0)
    all_History_Application = all_History_Application.reset_index(drop=True)
    all_History_Application = all_History_Application.rename(columns={'贷款类型':'历史贷款类型','贷款年金':'历史贷款年金',
                                                                    '信用额度':'历史信用额度','商品价格':'历史商品价格', })
    all_History_Application['历史贷款年金'] =all_History_Application['历史贷款年金'].fillna(0)
    all_History_Application['贷款期数'] =all_History_Application['贷款期数'].fillna(0)
    all_History_Application['首付金额'] =all_History_Application['首付金额'].fillna(0)
    all_History_Application['首付率'] =all_History_Application['首付率'].fillna(0)
    all_History_Application['陪同申请人'] =all_History_Application['陪同申请人'].fillna(99)
    all_History_Application['首付时间'] =all_History_Application['首付时间'].fillna(0)
    all_History_Application['产品组合'] =all_History_Application['产品组合'].fillna(99)
    all_History_Application = all_History_Application.drop(columns=['利率1','利率2'])
    记录', '是否为当天最后一次申请记录', 
                                     '贷款用途', '合同状态', '付款方式', '拒绝原因', '陪同申请人', '客户类型', '商品类别', 
                                     '组合类型', '产品类型', '获客渠道', '行业', '收益级别', '产品组合']
    all_History_Application_category_feature, _ = encode_category_feature(all_History_Application, history_application_category_cols)
    
    # 连续型变量的统计特征  
    all_History_Application_groupby = all_History_Application.loc[:, ["申请编号",'历史贷款年金', '申请额度', '信用额度', '首付金额',
                                                                      '商品价格', '首付率', '历史贷款授信距本次申请时间', 
                                                                      '贷款期数', '首付时间']].groupby(by=["申请编号"],as_index=False)
    
    all_History_Application_count = all_History_Application_groupby.count().rename(columns={"贷款期数": "count_贷款期数"})
    all_History_Application_sum =all_History_Application_groupby.sum().rename(columns={"历史贷款年金": "sum_历史贷款年金",
                                                                                       "申请额度": "sum_申请额度",
                                                                                       "信用额度":"sum_信用额度", 
                                                                                       "首付金额":"sum_首付金额",
                                                                                       "商品价格":"sum_商品价格",
                                                                                       "首付率": "sum_首付率",
                                                                                       "历史贷款授信距本次申请时间": "sum_历史贷款授信距本次申请时间",
                                                                                       "贷款期数": "sum_贷款期数",
                                                                                       "首付时间": "sum_首付时间"})
    all_History_Application_max =all_History_Application_groupby.max().rename(columns={"历史贷款年金": "max_历史贷款年金",
                                                                                           "申请额度": "max_申请额度",
                                                                                           "信用额度":"max_信用额度", 
                                                                                           "首付金额":"max_首付金额",
                                                                                           "商品价格":"max_商品价格",
                                                                                           "首付率": "max_首付率",
                                                                                           "历史贷款授信距本次申请时间": "max_历史贷款授信距本次申请时间",
                                                                                           "贷款期数": "max_贷款期数",
                                                                                           "首付时间": "max_首付时间"})

    all_History_Application_min =all_History_Application_groupby.min().rename(columns={"历史贷款年金": "min_历史贷款年金",
                                                                                       "申请额度": "min_申请额度",
                                                                                       "信用额度":"min_信用额度", 
                                                                                       "首付金额":"min_首付金额",
                                                                                       "商品价格":"min_商品价格",
                                                                                       "首付率": "min_首付率",
                                                                                       "历史贷款授信距本次申请时间": "min_历史贷款授信距本次申请时间",
                                                                                       "贷款期数": "min_贷款期数",
                                                                                       "首付时间": "min_首付时间"})
    all_History_Application_mean =all_History_Application_groupby.mean().rename(columns={"历史贷款年金": "mean_历史贷款年金",
                                                                                       "申请额度": "mean_申请额度",
                                                                                       "信用额度":"mean_信用额度", 
                                                                                       "首付金额":"mean_首付金额",
                                                                                       "商品价格":"mean_商品价格",
                                                                                        "首付率": "mean_首付率",
                                                                                       "历史贷款授信距本次申请时间": "mean_历史贷款授信距本次申请时间",
                                                                                       "贷款期数": "mean_贷款期数",
                                                                                       "首付时间": "mean_首付时间"})
    all_History_Application_std =all_History_Application_groupby.std().rename(columns={"历史贷款年金": "std_历史贷款年金",
                                                                                       "申请额度": "std_申请额度",
                                                                                       "信用额度":"std_信用额度", 
                                                                                       "首付金额":"std_首付金额",
                                                                                       "商品价格":"std_商品价格",
                                                                                       "首付率": "std_首付率",
                                                                                       "历史贷款授信距本次申请时间": "std_历史贷款授信距本次申请时间",
                                                                                       "贷款期数": "std_贷款期数",
                                                                                       "首付时间": "std_首付时间"})
    all_History_Application_std['申请编号'] = all_History_Application_mean['申请编号'].copy()

#     all_History_Application_skew =all_History_Application_groupby.skew().rename(columns={"历史贷款年金": "skew_历史贷款年金",
#                                                                                        "申请额度": "skew_申请额度",
#                                                                                        "信用额度":"skew_信用额度", 
#                                                                                        "首付金额":"skew_首付金额",
#                                                                                        "商品价格":"skew_商品价格", 
#                                                                                        "首付率": "skew_首付率",
#                                                                                        "历史贷款授信距本次申请时间": "skew_历史贷款授信距本次申请时间",
#                                                                                        "贷款期数": "skew_贷款期数",
#                                                                                        "首付时间": "skew_首付时间"})
    all_History_Application_skew['申请编号'] = all_History_Application_mean['申请编号'].copy()
    
    df_list = [all_History_Application_count, all_History_Application_sum, all_History_Application_max, all_History_Application_min, 
               all_History_Application_mean, all_History_Application_std, all_History_Application_skew, all_History_Application_category_feature, ]
    ans = pd.DataFrame(all_History_Application['申请编号'].unique(), columns=['申请编号'])
    for df in df_list:
        ans = pd.merge(left=ans, right=df, on='申请编号', how='outer')
    print('shape:', ans.shape)
    return ans
    
import time
t0 = time.time()
history_application_feature = history_application_feature_li(train_history_application, A_history_application)  
print(time.time() - t0)

D:\software\anaconda\envs\universal\lib\site-packages\pandas\core\indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


category shape: (153006, 155)
shape: (153006, 218)
55.59946894645691


# history_payment

In [4]:
def history_payment_feature(train_history_payment, test_history_payment):
    data = pd.concat([train_history_payment, test_history_payment])
    data_groupby = data.groupby(['申请编号']).agg(['count', 'mean', 'min', 'max', 'std'])
    col_index = list(data_groupby.columns)
    col_name = [tmp[0] + '_' + tmp[1] for tmp in col_index]
    df = pd.DataFrame(data_groupby.values, columns=col_name)
    df['申请编号'] = data_groupby.index

    tmp_na = all_History_Application.loc[all_History_Application['历史贷款类型']!=all_History_Application['历史贷款类型']]
    tmp_not_na = all_History_Application.loc[all_History_Application['历史贷款类型']==all_History_Application['历史贷款类型']]
    tmp_na = pd.merge(tmp_na,all_train_base,left_on='申请编号',right_on='申请编号',how='left')
    tmp_na = tmp_na.drop(['历史贷款类型'],axis=1)
    tmp_na = tmp_na.rename(columns={'贷款类型':'历史贷款类型'})
    all_History_Application0 = tmp_na.append(tmp_not_na)
    all_History_Application0['历史贷款类型'] = all_History_Application0['历史贷款类型'].apply(lambda x:str(x) if x==x else x)
    
        
    drop_cols = ['历史申请编号_count', '历史申请编号_mean', '历史申请编号_min', '历史申请编号_max', '历史申请编号_std']
    for col in df.columns:
        if '_count' in col and col != '贷款年金_count':
            drop_cols.append(col)
    df.drop(columns=drop_cols, inplace=True)
    print('payment shape:', df.shape)
    return df

paymeny_feature = history_payment_feature(train_history_payment, A_history_payment)
print(paymeny_feature.shape)


payment shape: (153309, 25)
(153309, 25)


# application

In [5]:
# def get_application_feature(train_application, test_application):
#     data = pd.concat([train_application, test_application])
#     data['工作日期距申请日期天数_01flag'] = data['工作日期距申请日期天数'].apply(lambda x: 0 if x >0 else 1)
#     data['贷款申请前1个季度内征信查询次数_01flag'] = data['贷款申请前1个季度内征信查询次数'].apply(lambda x: 1 if x > 0 else 0)
#     data['贷款申请前1天内征信查询次数_01flag'] = data['贷款申请前1天内征信查询次数'].apply(lambda x: 1 if x > 0 else 0)
#     data['贷款申请前1小时内征信查询次数_01flag'] = data['贷款申请前1小时内征信查询次数'].apply(lambda x: 1 if x > 0 else 0)
#     print("application shape:", data.shape)
#     return data

# application_feature = get_application_feature(train_application, A_application)


application shape: (161511, 35)


# personas

In [6]:
def get_presonas_feature(train_application, test_application):
    data = pd.concat([train_personas, A_personas])
    data['孩子个数'] = data['孩子个数'].apply(lambda x: x if x < 3 else 3)
    data['客户收入'] = data['客户收入'].apply(lambda x: x if x < 240000 else 240000)
    data['社交圈违约信息1_2'] = data['社交圈违约信息1_2'].apply(lambda x: x if x < 20 else 20)
    print("personas shape:", data.shape)
    return data
personas_features = get_presonas_feature(train_personas, A_personas)

personas shape: (161511, 32)


In [7]:
tabel_list = [application_feature, personas_features, history_application_feature, paymeny_feature, train_label]
ans = application_feature['申请编号'].copy()
for table in tabel_list:
    ans = pd.merge(left=ans, right=table, on='申请编号', how='outer')
print('feature shape:', ans.shape)

feature shape: (161511, 308)


In [9]:
import joblib
joblib.dump(ans, '../data/feature_308')

['../data/feature_308']

In [10]:
ans['标签'].fillna(-1, inplace=True)

In [11]:
ans['标签'].value_counts()

 0.0    119000
-1.0     21511
 1.0     21000
Name: 标签, dtype: int64

In [19]:
test = ans[ans['标签'] == -1].copy()
test_id = test['申请编号'].copy()
test.drop(columns=['标签', '申请编号'], inplace=True)
print('test shape:', test.shape)
train = ans[(ans['标签'] == -0) | (ans['标签'] == 1)]
label = train['标签'].copy()
print('label shape:', label.shape)
train.drop(columns=['标签', '申请编号'], inplace=True)
print('train shape:', train.shape)


test shape: (21511, 306)
label shape: (140000,)
train shape: (140000, 306)


D:\software\anaconda\envs\universal\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [20]:
def train_model_lgb(train, label, test):
    def ks(y_label, pred):
        fpr, tpr, thresholds= metrics.roc_curve(y_label, pred)
        ks = max(tpr-fpr)
        return 'ks', ks, True

    x_train, x_test, y_train, y_test = model_selection.train_test_split(train, label, train_size=0.80, random_state=345)

    model = lgb.LGBMClassifier(random_seed=2019, n_jobs=-1, objective='binary',
                               learning_rate=0.005, n_estimators=60000, num_leaves=31, max_depth=-1,
                               min_child_samples=3, min_child_weight=1, subsample_freq=1, 
                               subsample=0.7, colsample_bytree=0.7, reg_alpha=1, reg_lambda=5)
    
    model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_test, y_test)], eval_metric=ks,\
              verbose=500, early_stopping_rounds=500)
    model.n_estimators = model.best_iteration_
    pre_train = model.predict_proba(x_train)[:, 1]
    pre_test = model.predict_proba(x_test)[:, 1]
    pre_evl = model.predict_proba(test)[:, 1]
    
    print('train ks:', ks(y_train, pre_train)[1])
    print('validation ks:', ks(y_test, pre_test)[1])
    return model, pre_train, pre_test, pre_evl

In [21]:
model, pre_train, pre_test, pre_evl = train_model_lgb(train, label, test)

Training until validation scores don't improve for 500 rounds.
[500]	training's binary_logloss: 0.376923	training's ks: 0.380095	valid_1's binary_logloss: 0.385445	valid_1's ks: 0.334776
[1000]	training's binary_logloss: 0.362922	training's ks: 0.412874	valid_1's binary_logloss: 0.377825	valid_1's ks: 0.348538
[1500]	training's binary_logloss: 0.354113	training's ks: 0.435832	valid_1's binary_logloss: 0.374967	valid_1's ks: 0.356882
[2000]	training's binary_logloss: 0.34702	training's ks: 0.456896	valid_1's binary_logloss: 0.373649	valid_1's ks: 0.358547
[2500]	training's binary_logloss: 0.340767	training's ks: 0.478107	valid_1's binary_logloss: 0.372888	valid_1's ks: 0.361064
[3000]	training's binary_logloss: 0.335089	training's ks: 0.496297	valid_1's binary_logloss: 0.372371	valid_1's ks: 0.362364
[3500]	training's binary_logloss: 0.329741	training's ks: 0.513948	valid_1's binary_logloss: 0.372098	valid_1's ks: 0.363047
[4000]	training's binary_logloss: 0.324594	training's ks: 0.5305

In [ ]:
ans.to_csv('../submit/pre_test_9.csv', columns=['申请编号', 'result'], index=None, header=None, encoding='UTF-8')